This workbook provides a template for estimating SD models in Vensim using Bayesflow package. The code is structured so that all key architectural choices are made at the top (including Vensim model features) and the code can run for any model after setting those initials. The running of the full code is also functionalized, so it is easy to run more complex analysis and save key outputs as desired.

In [ ]:
''' Defining key structural parameters of the analysis '''
import numpy as np

# Vensim model and parameters
vensimInputs={
    'venModelName': 'RandomWalk2', #name of the vensim model (should be .vmpx for dll, and .mdlx for script-based)
    'venVOCFile' : 'RandomWalk2.voc',  #A .voc or .out file from whichc estimated parameter names and ranges can be read (must include upper and lower bounds); if not available, leave as '' and populate 'venParameters' below
    'SeedVar' : 'NSeed', # Name of the variable in vensim that controls the noise seed; default is 'NSeed'
    'venParameters': {"Drift": (-1, 1), "InitS": (0, 10), "NoiseStd": (0, 0.5), "MeasStd" : (0, 0.5)},
    'shortHandNames': [r"$Drift$", r"$S0$", r"$\sigma_P$", r"$\sigma_M$"], #Shorthand for parameter names to be used in graphs; leave empty, [], so it is populated based on actual vensim names
    'outputs': ['State Obs'],  #vensim output variables to be recorded and trained on
    'out_labels' : ['State Obs'], # labeles for output variables
    'out_units' : ['Dimensionless'], # units for output variables
    'constants' : ['Final Time'], #Name of constants to be set before start of the simulations
    'constant_vals' : [99],  #Values of constants to be set for all simulations in a context
    'Time_points' : np.arange(100), #All times for which data points should be extracted; should be consistent with Final Time- Start Time
    'useDll' : True, # Using DLL (venpy) or script based (VST) connection with Vensim
    'use_ven_multi' : True, #whether to use multicore or single core vensim
    'vengine_path': "C:/Program Files/Vensim/vendss64MC.exe",  # Change this to your Vengine file path!
    'vensim_path': "C:/Program Files/Vensim/vendss64.exe",  # Change this to your Vensim file path!
}


# Bayesflow inputs and parameters
BFInputs={
    #summary network settings
    'sum_network_type' : 'sequence', # use sequence or transformer
    'num_conv_layers' : 2, # Number of convolution layers for summary network
    'lstm_units': 32, # number of lstm units in summary network
    'bidirectional': False, #whether to use bidirectional or unidirectional in summary network
    # transformer only settings
    'num_heads' : 4, # number of heads for transformer
    'key_dim' : 32, #key dimensionality for transformer
    'num_attention_blocks' : 2, # dimensionality of attention blocks for transformer
    'num_dense' : 2, # number of dense_fc for transformer
    # inference network settings
    'num_coupling_layers' : 2, # Number of coupling layers for inference network
    # configurator settings
    'float16' : False, #whether to use float16 or float32 for NN training; given the range of float16 it is a good idea to log-transform the results if using this option
    'log_outputs': False, # whether to log-trasform the output data or use actual values
    'normalize_params' : False, #whether to normalize parameters or use raw values
    'remove_extremes' : True, #whether to remove any dataset with nan or inf values, or replace them with fixed values below
    'nan_rplc' : -50000, # The value to replace nans in the simulations with; only active if not removing extremes
    'posinf_rplc':  51000, # The value to replace positive infinity with; only active if not removing extremes
    'neginf_rplc' : -51000, # The value to replace negative infinity with; only active if not removing extremes
    # training settings
    'learning_rate': 0.0005, # learning rate for training of networks
    'TrainType' : 'rounds', # Use 'offline', 'online', or 'rounds', with default (any other text) leading to rounds
    'num_rounds' : 5, # number of rounds for both offline and round-based training
    'sims_per_round' : 1024, # number of sims per round; normalized for comparability
    'epochs' : 20, # number of epochs per round, or in case of online training, total number of epochs (each with iterations of batchs)
    'batch_size' : [32], # batch sizes for off(/on)line training rounds; for round based only the first size is used
    # validation settings
    'num_valid_sims' : 1000, # number of validation sims to use
    # other settings
    'modelName' : 'SEIRb', # Used for naming the model in Bayesflow, though plays little roll otherwise
    'summary_dim' : 5, # Number of summary Dimensions
    'trainer_memory' : True, # whether to hold a memory of simulations for later use
    'save_checkpoint' : False, # whether to save the amortizer. Should modify the path below to point to the correct directory
    'checkpoint_path': 'checkpoints/', # put a string for the folder to store trained amortizer, loss history, and optional memory
    'reuse' : False, # whether the optimizer state should be kept for reusing; reuse may lead to very slow learning
    # manual summary stat settings
    'manual_sum_stats' : False, #whether to use manual summary stats for residual covariance
    'window_sum_stats' : 11, # time window for savitzky-golay filter
    'poly_sum_stats' : 2, #polynomial order for the filter
    'lags_sum_stats' : [0,3,10], # lags used for calculating covariances
}

resInputs={
    'graph_all': True
}
resInputs={
    'scnr_nm' : 'RW_Overkill', #scenaior name used for saving graphs etc
    'save_graphs' : True, #whether to save the graphs as jpeg
    'valid_size': 500, #size of the validation dataset after training
    'sample_size': 1000, #size of posterior samples for each of validation data
    'rcvr_prcnl' : 80, # percentiles for recovery plots
    'graph_prior' : resInputs['graph_all'], #whether to graph priors
    'graph_losses' : resInputs['graph_all'], #whether to graph loss history
    'graph_latent' : resInputs['graph_all'], #whether to graph latesnt space after training
    'graph_SBC' : resInputs['graph_all'], #graph classical SBC using training data
    'graph_ecdf' : resInputs['graph_all'], #whether to graph SBC ECDF charts for new validation SBC
    'graph_recovery' : resInputs['graph_all'], #graph credible interval recovery
    'graph_z_contract' : resInputs['graph_all'], #graph z-scores and contraction for outcomes
    'graph_CI' : resInputs['graph_all'], #graph CI recovery quality
    'posterior_predictive' : resInputs['graph_all'], #resInputs['graph_all'], #graph posteriors based on a single dataset
    'ppsize' : 1, # number of single sample posteriors to draw and demonstrate
}
    


In [ ]:
# import various needed libraries 
#criticals, used in all cases

import bayesflow as bf
import datetime
from functools import partial
import pandas as pd
import bayesflow.diagnostics as diag
from bayesflow.amortizers import AmortizedPosterior
from bayesflow.networks import InvertibleNetwork, SequenceNetwork, TimeSeriesTransformer
from bayesflow.simulation import GenerativeModel, Prior, Simulator
from bayesflow.trainers import Trainer
from scipy import stats
from scipy.stats import nbinom
import matplotlib.pyplot as plt
import os
import shutil
from ctypes import c_float
import venpy
from vst import *
from scipy.signal import savgol_filter
import seaborn as sns
from sklearn.neighbors import KernelDensity


In [ ]:
# function to parse out the vensim parameters from .voc or .out files
def parse_voc_file(filepath):
    """
    Parses a .voc file and extracts parameters and their ranges, with correct splitting.
    Args:
    - filepath (str): Path to the .voc file.
    Returns:
    - dict: Dictionary with parameter names as keys and their ranges as values.
    """
    variables = {}
    with open(filepath, 'r') as file:
        start_processing = False
        for line in file:
            if line.startswith(':'):
                start_processing = True
                continue

            if start_processing and line.strip():
                # Extract parts of the line
                parts = line.strip().split('<=')
                param = parts[1].strip()  # The parameter name is the second part
                lower_bound = float(parts[0].strip())
                upper_bound = float(parts[2].strip())
                variables[param] = (lower_bound, upper_bound)

    return variables
    

# define a prior for the generative model.
def model_prior(variables):
    """Generates a random draw from the joint prior.
    Args: 
    - parameter names and ranges
    Returns: 
    - draws from the priors
    """
    
    # Generate random numbers and create a NumPy array
    random_values = np.array([
        np.random.uniform(low, high) for name, (low, high) in variables.items()
    ])
    return random_values



In [ ]:

# function for getting variables from bayesflow and creating the vensim sensitivity file required
def create_tab_delimited_file(
    parameter_matrix, parameter_names, filename, seed_start, SeedVar='NSeed'
):
    """
    Creates a tab-delimited text file from a NumPy array of parameters, including
    NSeed and additional constant columns.

    Args:
        parameter_matrix (numpy.ndarray): A 2D array of parameters, with shape (#simulations, #parameters).
        parameter_names (list): A list of parameter names, corresponding to the columns of the array.
        constant_names (list): A list of names for the constant columns.
        constant_values (list or numpy.ndarray): A list or array of constant values, with length equal to the number of constant names.
        filename (str): The name of the text file to create.
    """

    with open(filename, "w") as f:
        # Write the header row with all column names
        header = [SeedVar] + parameter_names
        f.write("\t".join(header) + "\n")

        # Write the parameter values, NSeed, and constants for each simulation
        for i in range(len(parameter_matrix)):
            simulation_values = np.insert(parameter_matrix[i], 0, i + 1 + seed_start)  # Insert NSeed at the beginning
            f.write("\t".join(map(str, simulation_values)) + "\n")


#This helper furnction will be used for creating the vensim input files (lst, vsc, etc)
def write_strings_to_file(strings, filename):
    """
    Writes a set of strings to a text file, each on a new line.

    Args:
        strings: A set of strings to write.
        filename: The name of the text file to write to.
    """

    with open(filename, "w") as f:
        for string in strings:
            f.write(f"{string}\n")  # Write each string with a newline character

def extract_sensitivities_sorted(model, variable_names, all_times, sensitivity_count, SeedVar='NSeed'):
    """
    Extracts and sorts sensitivity data for given variable names and times based on 'NSeed' value.
    
    Args:
    - model: The model object with the 'getsensitivity' method.
    - variable_names (list of str): The names of the variables for which to extract data.
    - all_times (list): List of all time points in the sensitivity simulation.
    - sensitivity_count (int): The number of sensitivity simulations conducted.
    
    Returns:
    - numpy.ndarray: Array of dimensions (#Sensitivity, #Times, #Variables), sorted by 'NSeed'.
    """
    # Extract the NSeed values for sorting
    vec, length = model.getsensitivity(SeedVar, 0)
    nseed_values = np.ctypeslib.as_array(vec, shape=(length,))
    
    # Check that the number of NSeed values matches the expected sensitivity count
    if length != sensitivity_count:
        raise ValueError(f"Expected sensitivity count {sensitivity_count}, but got {length}")
    
    # Sort the indices of simulations based on NSeed values
    sorted_indices = np.argsort(nseed_values)

    # Initialize an array to hold the sorted sensitivities
    sorted_sensitivities = np.empty((sensitivity_count, len(all_times), len(variable_names)))

    # Extract and sort data for each variable and time point
    for i, var_name in enumerate(variable_names):
        for j, time in enumerate(all_times):
            # Get the sensitivity vector for the current variable at the current time
            vec, _ = model.getsensitivity(var_name, time)
            sensitivities = np.ctypeslib.as_array(vec, shape=(sensitivity_count,))

            # Sort the sensitivities based on sorted_indices
            sorted_sensitivities[:, j, i] = sensitivities[sorted_indices]

    return sorted_sensitivities



# helper function to get VST to extract data from tab file of sensitivity runs, after sorting based on NSeed
def extract_simulation_output(filepath, output_variables, SeedVar='NSeed'):
    # Read the tab-delimited file
    df = pd.read_csv(filepath, delimiter='\t')

    # Separate the parameter rows (where 'Time' is NaN) and output rows (where 'Time' is not NaN)
    df_params = df[df['Time'].isna()]
    df_output = df[df['Time'].notna()]
    # Sort the parameter dataframe by 'NSeed' and get the sorted simulation IDs
    df_params_sorted = df_params.sort_values(by=SeedVar)
    sorted_simulations = df_params_sorted['Simulation'].values

    # Get unique time points
    times = df_output['Time'].unique()

    # Initialize an array to hold the extracted and sorted data
    sorted_extracted_data = np.empty((len(sorted_simulations), len(times), len(output_variables)))

    # Extract and sort data for each simulation and output variable
    for i, sim in enumerate(sorted_simulations):
        sim_df = df_output[df_output['Simulation'] == sim]
        for j, var in enumerate(output_variables):
            sorted_extracted_data[i, :, j] = sim_df[var].values

    return sorted_extracted_data




# calculating manual summary stats and appending to data

def calculate_residuals_covariance(data, window_length, polyorder, lags):
    n_batch, n_points, n_series = data.shape
    all_results = []
    
    # Determine the size for #batch_series_covariance based on lags and n_series
    max_cov_size = len(lags) * n_series + 1  # +1 for length at the end

    for batch_index in range(n_batch):
        current_batch = data[batch_index, :, :]
        smoothed_batch = np.array([savgol_filter(current_batch[:, series], window_length, polyorder) 
                                   for series in range(n_series)]).T
        residuals = current_batch - smoothed_batch
        residuals[np.isnan(residuals)] = 0  # Replace NaNs with 0 for covariance calculation
        
        # Initialize an array for covariances, considering the adjusted output dimension
        batch_covariances = np.zeros((max_cov_size, n_series))
        
        for i in range(n_series):
            covariances = []
            for j in range(n_series):  # Compute for all pairs
                for lag in lags:
                    if lag==0:
                        series_i, series_j = residuals[:, i], residuals[:, j]
                    else:
                        series_i, series_j = residuals[:-lag, i], residuals[lag:, j]

                    min_length = min(len(series_i), len(series_j))
                    series_i, series_j = series_i[:min_length], series_j[:min_length]
                    
                    if len(series_i) > 0 and len(series_j) > 0:
                        cov_ij = np.cov(series_i, series_j, ddof=0)[0, 1]
                        covariances.append(cov_ij)
                        
            # transform to sqrt
            covariances_sqr = np.sqrt(np.abs(covariances)) * np.sign(covariances)
            
            # Store the covariances and their length in the array
            covariances_length = len(covariances_sqr)
            batch_covariances[:covariances_length, i] = covariances_sqr
            batch_covariances[covariances_length, i] = covariances_length  # Store length at the end
        
        all_results.append(batch_covariances)
    
    
    return np.array(all_results)






In [ ]:

# Simulation model in vensim, connected through dll or scripts. Note that this function is given the params (which vary across simulations) and various hyper parameters and variables.
def vensimSimulationModel(params, venInputs, BFInputs):
    """Performs a batch of forward simulations from the SD model given a batch of random draws from the prior."""
    # First the setup of the function
    venModelPub = venInputs['venModelName']+'.vpmx' #name of the published model
    venModelNorm = venInputs['venModelName']+'.mdl' #name of the published model
    simSubDir='VensimRuns'  #name of subdirectory to do simulations in when using scripts
    venSensFile="SIRSens.txt"  #name the sensitivity file vensim needs for reading parameters to simulate; will be created below.
    venSaveList='sens.lst' #name of the savelist used in recording model outputs, to be created below
    venSensControl = "Sens.vsc" #name of the sensitivity control file, to be created below
    seed_start=np.random.randint(0, 10000000) #set a new start point for random numbers creating the noise streams
    #Vensim parameter names (need exact text), order is important and should match those of the 'prior' parameters
    ven_param_names = list(venInputs['venParameters'].keys())
    #Vensim constants that are the same across all runs in a batch and match the inputs of the function
    ven_const_names = venInputs['constants']
    #Values of vensim constants
    Constants = venInputs['constant_vals']
    # Simulation outputs used in the analysis
    output_vars = venInputs['outputs']  # output variable names in the order expected
    # Number of data points to be extracted
    Times = venInputs['Time_points'] 
    # Noise seed variable name
    venSeedVar=venInputs['SeedVar']

 
    # create vensim savelist for sensitivity
    write_strings_to_file(output_vars, venSaveList)

    # create sensitivity control file for vensim
    write_strings_to_file(['200,F,1234,' + os.getcwd() + '/' + venSensFile + ',0'], venSensControl)

    
    # create the sensitivity file for vensim. This could include other constant-across-sim parameter changes not passed in setval
    create_tab_delimited_file(params,ven_param_names, venSensFile, seed_start, SeedVar=venSeedVar)

    if venInputs['useDll']:
        # Read the model
        venmodel = venpy.load(venModelPub)
        # set general parameters
        i = 0
        for varNm in ven_const_names:
            venmodel[varNm] = Constants[i]
            i = i+1
        # do the simulations
        venmodel.run(runname='sens',sensitivity=[venSensControl,venSaveList])
        # get the data
        sim_data = extract_sensitivities_sorted(venmodel, output_vars, Times , params.shape[0], SeedVar=venSeedVar)
    else:
        # Removes the simulation subdirectory and its contents if it exists."""
        subdirectory_path = "./" + simSubDir
        if os.path.exists(subdirectory_path) and os.path.isdir(subdirectory_path):
            try:
                # Use shutil.rmtree for robust deletion
                shutil.rmtree(subdirectory_path)
                print(f"Subdirectory '{subdirectory_path}' and its contents deleted successfully.")
            except OSError as error:
                print(f"Error removing subdirectory '{subdirectory_path}': {error}")
    
        
        # creating tuple for setvals in the script for vensim. Note the wrapping of function inputs here.
        setval_tuples = tuple(zip(ven_const_names, Constants))

        # Global control inputs for vensim
        timelimit = 300  # Time limit (in seconds) for automatic checking/monitoring of runs
        if venInputs['use_ven_multi']:
            vgpath = venInputs['vengine_path'] 
        else:
            vgpath = venInputs['vensim_path'] 
        
        # Setup the control file for the sensitivity run
        # Controlfile inputs. The empty fields could be potentially removed, but kept here for generalizeability
        cf = {
            'basename': 'sens', #Note this name becomes the name of the sensitivity simulation
            'simcontrol': {
                "model": venModelNorm, #This is the name of the model in the root directory
                "data": [], 
                "payoff": "", 
                "sensitivity": venSensControl, #This is the name of sensitivity control file; in it you should point to venSensFile in the correct directory
                "optparm": "", 
                "changes": [], 
                "savelist": "",
                "setvals": [],
                "senssavelist": venSaveList #This savelist should include all the variables the model should give to bayesflow
            }, 
            'runcmd': '', 
            'savecmd': 'SENS2FILE|!|!|T'    #This is the command file for exporting sensitivity data; you may need to change this if frequency of saving of data in vensim is different from what you need in bayesflow
        }
        
        # Create logfile; not critical
        log = f"{os.getcwd()}/log.txt"
        
        # Run the vensim command; this is where actual simulations happen
        sens = Script(cf, "", log, setvals=setval_tuples, simtype='sf')
        #sens.compile_script(vgpath, log, subdir=simSubDir, timelimit=timelimit, check_funcs=[check_output])
        sens.compile_script(vgpath, log, subdir=simSubDir)
        
        # Take the outputs from tab and put into the desired format
        
        file_path = simSubDir + "/" + cf['basename'] + ".tab" # file path for the results of simulations
        
        # Extract data
        sim_data = extract_simulation_output(file_path, output_vars)
    
        # Remove helper vensim files to reduce clutter from baseline folder (but leaves intact the main simulation folder for inspection)
        for filename in (venSensFile,venSaveList, venSensControl):
            filepath = os.path.join(os.getcwd(), filename)  # Construct full path
            try:
                os.remove(filepath)  # Attempt to delete the file
                print(f"Deleted file: {filename}")
            except FileNotFoundError:
                print(f"File not found: {filename}")
            except OSError as error:
                print(f"Error deleting file {filename}: {error}")

    # calculating manual summary stats and inserting into time series format to bypass dimensionality restrictions
    if BFInputs['manual_sum_stats']:
        summary_stats=calculate_residuals_covariance(sim_data, BFInputs['window_sum_stats'], BFInputs['poly_sum_stats'], BFInputs['lags_sum_stats'])


        # Determine new dimensions
        n_batch, series_len, n_series = sim_data.shape
        _, batch_series_covariance_plus_one, _ = summary_stats.shape
        
        # Initialize new_data with the correct shape
        new_series_len = series_len + batch_series_covariance_plus_one
        new_data = np.zeros((n_batch, new_series_len, n_series))
        
        # Copy original data into new_data
        new_data[:, :series_len, :] = sim_data
        
        # Append residual_covariance at the end of each series
        new_data[:, series_len:, :] = summary_stats
        
        # Update sim_data to the new array
        sim_data = new_data

    return sim_data



# configurator helps log-transform, change format to float 32 or 16, and remove problematic sim values
def configure_input(forward_dict, BFInputs):
    """Function to configure the simulated quantities (i.e., simulator outputs)
    into a neural network-friendly (BayesFlow) format.
    """
    # Prepare placeholder dict
    out_dict = {}

    if BFInputs['manual_sum_stats']:
               
        # Step 1: Determine length of summary stats from the last element in the second 'column'
        x = int(forward_dict["sim_data"][0, -1, 0])  # Assuming x is the same for all, convert to int due to float storage
        
        # Step 2: Extract summary_stats without using a for loop
        summary_stats_all = forward_dict["sim_data"][:, -x-1:, :]

        # Step 3: falttening summary stats
        summary_stats = summary_stats_all.reshape(summary_stats_all.shape[0], summary_stats_all.shape[1]*summary_stats_all.shape[2])
        
        # Step 4: add summary stats as direction conditions       
        if BFInputs['float16']:
            out_dict['direct_conditions'] = summary_stats.astype(np.float16)
        else: 
            out_dict['direct_conditions'] = summary_stats.astype(np.float32)

        showDiag=False
        if showDiag:
            data=forward_dict["sim_data"]
            n_batch, n_points, n_series = data.shape
            plt.figure(figsize=(5,3))
            for i in range(n_batch):
                plt.plot(data[i,:,0], label=f'b {i+1}')
    
            plt.legend()
            plt.show()
    
            print(out_dict['direct_conditions'])



    # Convert data to logscale
    if BFInputs['float16']:
        if BFInputs['log_outputs']:
            outdata = np.log1p(forward_dict["sim_data"]).astype(np.float16)
        else:
            outdata = forward_dict["sim_data"].astype(np.float16)
        
        if BFInputs['normalize_params']:
            # z-standardize with previously computed means
            params = (forward_dict["prior_draws"] - prior_means) / prior_stds
        else: 
            params = forward_dict["prior_draws"]
        # turn to float 16
        params = params.astype(np.float16)

    else: 
        if BFInputs['log_outputs']:
            outdata = np.log1p(forward_dict["sim_data"]).astype(np.float32)
        else:
            outdata = forward_dict["sim_data"].astype(np.float32)
        
        if BFInputs['normalize_params']:
            # z-standardize with previously computed means
            params = (forward_dict["prior_draws"] - prior_means) / prior_stds
        else: 
            params = forward_dict["prior_draws"]
            
        # turn to float 32
        params = params.astype(np.float32)

    if BFInputs['remove_extremes']:
        # Remove a batch if it contains nan, inf or -inf
        idx_keep = np.all(np.isfinite(outdata), axis=(1, 2))
        if not np.all(idx_keep):
            print("Invalid value encountered...removing from batch")
        # Add to keys
        out_dict["summary_conditions"] = outdata[idx_keep]
        out_dict["parameters"] = params[idx_keep]
    else:
        outdataclean=np.nan_to_num(outdata, nan=BFInputs['nan_rplc'] , posinf=BFInputs['posinf_rplc'] , neginf= BFInputs['neginf_rplc'])
        out_dict['summary_conditions'] = outdataclean
        out_dict["parameters"] = params



    if BFInputs['sum_network_type']=='transformer':
        x = out_dict['summary_conditions']
        
        # add time encoding to the data x
        batch_size, num_timesteps, data_dim = x.shape
        time_encoding = np.linspace(0, 1, num_timesteps)
        time_encoding_batched = np.tile(time_encoding[np.newaxis, :, np.newaxis], (batch_size, 1, 1))
        
        out_dict['summary_conditions'] = np.concatenate((x, time_encoding_batched), axis=-1)

    

    return out_dict

In [ ]:
# creating graphing functions
# k-percentile ranges for conf intvls
def centered_k_percentile_range(data, k, axis=None):
    """
    Calculates the centered k-percentile range of a dataset.

    Args:
        data: A list or NumPy array of numerical data.
        k: The percentile value (as a percentage, e.g., 25 for 25th percentile).
        axis: The axis along which to calculate the range (default: None for a flattened array).

    Returns:
        The centered k-percentile range (the difference between the upper and lower k-percentiles).
    """
    # Ensure data is a NumPy array for efficient operations
    data = np.asarray(data)

    lower_percentile = np.percentile(data, k, axis=axis)
    upper_percentile = np.percentile(data, 100 - k, axis=axis)
    return upper_percentile - lower_percentile

# CI consistency graph
def analyze_confidence_intervals_combined(input_sample, posterior_sample, param_names, N=20):
    num_params = input_sample.shape[1]
    num_columns = min(num_params, 6)
    num_rows = int(np.ceil(num_params / num_columns))
    
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(num_columns * 5, num_rows * 5))
    if num_params > 1:
        axes = axes.flatten()  # Flatten the axes array for easy indexing
    else:
        axes = [axes]  # Wrap in list if only one plot

    for param_index in range(num_params):
        ax = axes[param_index]
        param_name = param_names[param_index]
        ground_truths = input_sample[:, param_index]
        posteriors = posterior_sample[:, :, param_index]

        fractions = np.zeros(N)

        for i in range(1, N + 1):
            lower_percentile = 50 - 50 * i / N
            upper_percentile = 50 + 50 * i / N

            ci_lower = np.percentile(posteriors, lower_percentile, axis=1)
            ci_upper = np.percentile(posteriors, upper_percentile, axis=1)

            within_interval = (ground_truths >= ci_lower) & (ground_truths <= ci_upper)
            fractions[i - 1] = np.mean(within_interval)

        ax.plot(np.arange(1, N + 1) / N, fractions, marker='o', linestyle='-', color="#8f2727", markersize=8, alpha=0.9)
        ax.plot([0, 1], [0, 1], linestyle='dashed', color='black', linewidth=1.5, alpha=0.9)  # 45-degree reference line
        ax.set_title(f"{param_name}", fontsize=21)
        ax.set_xlabel("Interval Size", fontsize=19)
        ax.set_ylabel("Fraction Within Interval", fontsize=19)
        #ax.set_ylim(0, 1)
        ax.set_ylim([-0.05, 1.05])
        ax.tick_params(axis='both', which='major', labelsize=15)
        ax.tick_params(axis='both', which='minor', labelsize=15)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.grid(alpha=0.5)
        ax.grid(True)

    for i in range(num_params, num_rows * num_columns):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()
    return fig


def plot_posterior_2d_GT(
    posterior_draws,
    prior=None,
    prior_draws=None,
    param_names=None,
    params_ground_truth=None,  # New parameter
    height=3,
    label_fontsize=14,
    legend_fontsize=16,
    tick_fontsize=12,
    post_color="#8f2727",
    prior_color="gray",
    post_alpha=0.9,
    prior_alpha=0.7,
):
    assert len(posterior_draws.shape) == 2, "Shape of `posterior_samples` for a single data set should be 2 dimensional!"
    n_draws, n_params = posterior_draws.shape

    if prior is not None and prior_draws is None:
        draws = prior(n_draws)
        prior_draws = draws["prior_draws"] if type(draws) is dict else draws

    if param_names is None:
        param_names = [f"$\\theta_{{{i}}}$" for i in range(1, n_params + 1)]

    posterior_draws_df = pd.DataFrame(posterior_draws, columns=param_names)

    g = sns.PairGrid(posterior_draws_df, height=height)
    g.map_diag(sns.histplot, fill=True, color=post_color, alpha=post_alpha, kde=True)
    g.map_lower(sns.kdeplot, fill=True, color=post_color, alpha=post_alpha)

    if prior_draws is not None:
        prior_draws_df = pd.DataFrame(prior_draws, columns=param_names)
        g.data = prior_draws_df
        g.map_diag(sns.histplot, fill=True, color=prior_color, alpha=prior_alpha, kde=True, zorder=-1)
        g.map_lower(sns.kdeplot, fill=True, color=prior_color, alpha=prior_alpha, zorder=-1)

    if params_ground_truth is not None:
        # Add ground truth as vertical lines and dots
        for i, val in enumerate(params_ground_truth):
            g.axes[i, i].axvline(val, color='k', linestyle='--')
            for j in range(i):
                g.axes[i, j].plot(params_ground_truth[j], val, 'ko')
            for j in range(i+1, n_params):
                g.axes[j, i].plot(val, params_ground_truth[j], 'ko')

    if prior_draws is not None or prior is not None:
        handles = [Line2D([], [], color=post_color, lw=3, alpha=post_alpha),
                   Line2D([], [], color=prior_color, lw=3, alpha=prior_alpha)]
        g.fig.legend(handles, ["Posterior", "Prior"], fontsize=legend_fontsize, loc="center right")

    for i, j in zip(*np.triu_indices_from(g.axes, 1)):
        g.axes[i, j].axis("off")

    for i, j in zip(*np.tril_indices_from(g.axes, 1)):
        g.axes[i, j].tick_params(axis="both", which="major", labelsize=tick_fontsize)
        g.axes[i, j].tick_params(axis="both", which="minor", labelsize=tick_fontsize)

    for i, param_name in enumerate(param_names):
        g.axes[i, 0].set_ylabel(param_name, fontsize=label_fontsize)
        g.axes[len(param_names) - 1, i].set_xlabel(param_name, fontsize=label_fontsize)

    for i in range(n_params):
        for j in range(n_params):
            g.axes[i, j].grid(alpha=0.5)

    g.tight_layout()
    return g.fig


def plot_ppc(real_data, post_sims, y_labels, titles, logscale=False, color="#8f2727", figsize=(10, 6), font_size=14, cut_end=False):
    """
    Plots posterior predictive checks for all series as subplots in a single figure.

    Parameters:
    - real_data: np.ndarray, shape [time_len, num_series]
    - post_sims: np.ndarray, shape [num_batch, time_len, num_series]
    - y_labels: list of str, Y axis labels for each series
    - titles: list of str, titles for each subplot
    - logscale: bool, if True, Y axis will be in logarithmic scale
    - color: str, color for prediction intervals and median line
    - figsize: tuple, overall figure size
    - font_size: int, font size for text elements
    
    Returns:
    - f: plt.Figure, the combined figure with all subplots
    """
    plt.rcParams["font.size"] = font_size
    num_series = real_data.shape[1]
    if cut_end:
        x_len=int(post_sims.shape[1]-post_sims[0,-1,0]-1)
    else:
        x_len = post_sims.shape[1]
    f, axes = plt.subplots(num_series, 1, figsize=figsize, constrained_layout=True)
    
    if num_series == 1:
        axes = [axes]  # Ensure axes is iterable for a single subplot
    
    for i, ax in enumerate(axes):
        sims = np.array(post_sims)[:,:x_len,i]
        real_series_data = real_data[:x_len,i]
        
        qs_50 = np.quantile(sims, q=[0.25, 0.75], axis=0)
        qs_90 = np.quantile(sims, q=[0.05, 0.95], axis=0)
        qs_95 = np.quantile(sims, q=[0.025, 0.975], axis=0)
        
        ax.plot(np.median(sims, axis=0), label="Median simulated", color=color)
        ax.plot(real_series_data, marker="o", label="Data", color="black", linestyle="dashed", alpha=0.8)
        ax.fill_between(range(real_series_data.shape[0]), qs_50[0], qs_50[1], color=color, alpha=0.5, label="50% CI")
        ax.fill_between(range(real_series_data.shape[0]), qs_90[0], qs_90[1], color=color, alpha=0.3, label="90% CI")
        ax.fill_between(range(real_series_data.shape[0]), qs_95[0], qs_95[1], color=color, alpha=0.1, label="95% CI")
        
        ax.grid(color="grey", linestyle="-", linewidth=0.25, alpha=0.5)
        ax.spines["right"].set_visible(False)
        ax.spines["top"].set_visible(False)
        ax.set_xlabel("Time")
        ax.set_ylabel(y_labels[i])
        ax.set_title(titles[i])
        if logscale:
            ax.set_yscale("log")
        ax.legend(fontsize=font_size-2)
        ax.minorticks_off()
    f.show()
        
    return f

def estimate_map(samples):
    bw = 1.06 * samples.std() * samples.size ** (-1 / 5.)
    scores = KernelDensity(bandwidth=bw).fit(samples.reshape(-1, 1)).score_samples(samples.reshape(-1, 1))
    max_i = scores.argmax()
    map_i = samples[max_i]
    return map_i



In [ ]:
def run_full_analysis(vensimInputs, BFInputs, resInputs):

    # get the input parameters for priors distribution
    if vensimInputs['venVOCFile']:
        venParams=parse_voc_file(vensimInputs['venVOCFile'])
    else:
        venParams= vensimInputs['venParameters']
    #update vensimInputs
    vensimInputs['venParameters']=venParams
    
    # get shorthands for parameter names
    if vensimInputs['shortHandNames']:
        venParNames=vensimInputs['shortHandNames']
    else:
        venParNames=list(venParams.keys())
        
    # build te prior object
    prior = Prior(prior_fun=partial(model_prior, variables=venParams), param_names=venParNames)
    # test if it works and get its basic stats
    prior_means, prior_stds = prior.estimate_means_and_stds(1000)
    
    # We build the simulator using a partial function
    simulator = Simulator(batch_simulator_fun=partial(vensimSimulationModel, venInputs=vensimInputs, BFInputs=BFInputs))
    
    # creating the full generative model by bringing together the simulator and prior
    
    model = GenerativeModel(prior, simulator, name=BFInputs['modelName'])
    
    # As per default, the plot_prior2d function will obtain 1000 draws from the joint prior.
    if resInputs['graph_prior']:
        f = prior.plot_prior2d(n_samples=100)
        if resInputs['save_graphs']:
            f.savefig(resInputs['scnr_nm']+'_prior.svg', format='svg',dpi=300)
        

    if BFInputs['sum_network_type']=='sequence':
        # summary network, using a SequenceNetwork for time series
        summary_net = SequenceNetwork(summary_dim=BFInputs['summary_dim'], num_conv_layers=BFInputs['num_conv_layers'], lstm_units=BFInputs['lstm_units'],
                                      bidirectional=BFInputs['bidirectional'])
    elif BFInputs['sum_network_type']=='transformer':
        summary_net = TimeSeriesTransformer( len(vensimInputs['outputs']), attention_settings=dict(num_heads=BFInputs['num_heads'], key_dim=BFInputs['key_dim'], dropout=0.1), 
                                            dense_settings = None, use_layer_norm= True, num_dense_fc= BFInputs['num_dense'], summary_dim= BFInputs['summary_dim'], 
                                            num_attention_blocks=BFInputs['num_attention_blocks'], template_type='lstm', bidirectional=BFInputs['bidirectional'],
                                            template_dim=BFInputs['lstm_units'])

    
    # inference network
    inference_net = InvertibleNetwork(num_params=len(prior.param_names), num_coupling_layers=BFInputs['num_coupling_layers'])
    
    # amortizer definition glues together summary and inference networks
    amortizer = AmortizedPosterior(inference_net, summary_net, name=BFInputs['modelName']+"_amortizer")

    # fixing BFinputs for configurator
    configure_input_fixed = partial(configure_input, BFInputs=BFInputs)
    
    # defining the trainer
    trainer = Trainer(amortizer=amortizer, generative_model=model, configurator=configure_input_fixed, default_lr=BFInputs['learning_rate'], 
                      memory=BFInputs['trainer_memory'], checkpoint_path=BFInputs['checkpoint_path'])
    
    amortizer.summary()
    #define an output variable to record all desired outputs and output after running the big function
    runOutputs={}
    runOutputs['param_count']=amortizer.count_params()
    
    # reset the random number generator so different runs are comparable more easily
    RNG = np.random.default_rng(2023)
    start_time=time.time()
    # Main training block, choosing training mode and executing
    if BFInputs['TrainType']=='offline':
        for i in range(BFInputs['num_rounds']):
            offline_data = model(BFInputs['sims_per_round'])
            history = trainer.train_offline(offline_data, epochs=BFInputs['epochs'], batch_size=BFInputs['batch_size'][min(i,len(BFInputs['batch_size'])-1)], 
                                            save_checkpoint=BFInputs['save_checkpoint'], validation_sims=BFInputs['num_valid_sims'], reuse_optimizer=BFInputs['reuse'])
    elif BFInputs['TrainType']=='online':
            for i in range(BFInputs['num_rounds']):
                history = trainer.train_online(epochs=BFInputs['epochs'], iterations_per_epoch=round(BFInputs['sims_per_round']/BFInputs['batch_size'][min(i,len(BFInputs['batch_size'])-1)]), 
                                               batch_size=BFInputs['batch_size'][i], save_checkpoint=BFInputs['save_checkpoint'], validation_sims=BFInputs['num_valid_sims'], reuse_optimizer=BFInputs['reuse'])
    else: 
         history = trainer.train_rounds(BFInputs['num_rounds'], BFInputs['sims_per_round'], epochs=BFInputs['epochs'], 
                                       batch_size=BFInputs['batch_size'][0], save_checkpoint=BFInputs['save_checkpoint'], validation_sims=BFInputs['num_valid_sims'], reuse_optimizer=BFInputs['reuse'])
    
    end_time=time.time()
    # record training time
    runOutputs['train_time']=end_time-start_time
    
    # graph loss history
    if resInputs['graph_losses']:
        f = diag.plot_losses(history["train_losses"], history["val_losses"], moving_average=True)
        if resInputs['save_graphs']:
            f.savefig(resInputs['scnr_nm']+'_loss.svg', format='svg',dpi=300)
    runOutputs['val_loss']= np.mean(history["val_losses"][-10:])
    runOutputs['train_loss']= np.mean(history["train_losses"][-10:])
    
    # graph latent space
    if resInputs['graph_latent']:
        f = trainer.diagnose_latent2d()
        if resInputs['save_graphs']:
            f.savefig(resInputs['scnr_nm']+'_latent.svg', format='svg',dpi=300)
    
    # graph SBC basics
    if resInputs['graph_SBC']:
        f = trainer.diagnose_sbc_histograms()
        if resInputs['save_graphs']:
            f.savefig(resInputs['scnr_nm']+'_sbc.svg', format='svg',dpi=300)

    for i in range(resInputs['ppsize']):
        # Generate some validation data
        validation_sims = trainer.configurator(model(batch_size=resInputs['valid_size']))
        
        # Generate posterior draws for all simulations
        post_samples = amortizer.sample(validation_sims, n_samples=resInputs['sample_size'])
        
        #fsbc = diag.plot_sbc_histograms(post_samples, validation_sims["parameters"], param_names=prior.param_names)
        # Create ECDF plot
        if resInputs['graph_ecdf']:
            f = diag.plot_sbc_ecdf(post_samples, validation_sims["parameters"], param_names=prior.param_names,difference=True)
            if resInputs['save_graphs']:
                f.savefig(resInputs['scnr_nm']+ '_' + str(i) +'_ecdf.svg', format='svg',dpi=300)
        
        # calculate required percentiles
        k=(100-resInputs['rcvr_prcnl'])/2
        
        # graph recovery of parameters
        if resInputs['graph_recovery']:
            f = diag.plot_recovery(
                post_samples,
                validation_sims["parameters"],
                param_names=prior.param_names,
                uncertainty_agg=lambda data, axis=None: centered_k_percentile_range(data, k, axis=axis)
            )
            if resInputs['save_graphs']:
                f.savefig(resInputs['scnr_nm']+ '_' + str(i) +'_recovery.svg', format='svg',dpi=300)
        
        
        # Calculate MAD for each sample (second dimension)
        mad_per_sample = np.apply_along_axis(func1d=stats.median_abs_deviation, axis=1, arr=post_samples)
        
        # Average MAD across the first dimension
        average_mad = np.mean(mad_per_sample, axis=0)
        
        # Format and print each element individually
        formatted_average_mad = []
        for value in average_mad:
            formatted_average_mad.append(np.format_float_positional(value, precision=2, fractional=False))
        
        runOutputs['ave_MAD']=formatted_average_mad
        
        
        # Calculate posterior mean and standard deviation
        posterior_mean = np.mean(post_samples, axis=1)  # Mean across the samples (axis=1), results in (500, 4)
        posterior_std = np.std(post_samples, axis=1)  # Std across the samples (axis=1), results in (500, 4)
        
        # Calculate the post_z_score and contraction
        true_parameters = validation_sims['parameters']
        post_z_score = (posterior_mean - true_parameters) / posterior_std  # Results in (500, 4)
        post_contraction = 1-(np.var(post_samples, axis=1))/prior_stds**2
        runOutputs['z_mean'] = np.mean(post_z_score, axis=0)
        runOutputs['z_std'] = np.std(post_z_score, axis=0)
        runOutputs['cnt_mean'] = np.mean(post_contraction, axis=0)
        runOutputs['cnt_std'] = np.std(post_contraction, axis=0)
        
        # graph z scores and contractions
        if resInputs['graph_z_contract']:
            f= diag.plot_z_score_contraction(post_samples, validation_sims["parameters"], param_names=prior.param_names)
            if resInputs['save_graphs']:
                f.savefig(resInputs['scnr_nm']+ '_' + str(i) +'_zcont.svg', format='svg',dpi=300)
        
        # graph consistency of confidence intervals generated by the estimation
        if resInputs['graph_CI']:
            f= analyze_confidence_intervals_combined(validation_sims["parameters"], post_samples, prior.param_names)
            if resInputs['save_graphs']:
                f.savefig(resInputs['scnr_nm']+ '_' + str(i) +'_CI.svg', format='svg',dpi=300)

    # graph posteriors for a single run
    if resInputs['posterior_predictive']:
        # Get lower and upper bounds for parameters
        lower_bounds = np.array([bounds[0] for bounds in venParams.values()])
        upper_bounds = np.array([bounds[1] for bounds in venParams.values()])
        cut_end = BFInputs['manual_sum_stats']
        for i in range(resInputs['ppsize']):
            # Pick a single dataset as 'real' data
            real_data = trainer.configurator(model(batch_size=1))

            # Obtain a number of posterior draws given real data
            post_samples_raw = amortizer.sample(real_data, n_samples=BFInputs['num_valid_sims'])
            post_samples = np.clip(post_samples_raw, lower_bounds, upper_bounds)  # Ensure posterior samples are feasible

            ground_truth = real_data['parameters'][0]

            # Undo standardization to get parameters on their original (unstandardized) scales
            if BFInputs['normalize_params']:
                post_samples = prior_means + post_samples * prior_stds
                ground_truth = prior_means + ground_truth * prior_stds

            post_sims = simulator(post_samples)

            # Generate summary statistics and save as CSV
            qs_95 = np.quantile(post_samples, q=[0.025, 0.975], axis=0)
            qs_95_str = ['[{0:.3f} - {1:.3f}]'.format(qs_95[0, j], qs_95[1, j]) for j in range(len(prior.param_names))]
            meds = np.array(['{0:.3f}'.format(m) for m in np.median(post_samples, axis=0)])
            means = np.array(['{0:.3f}'.format(m) for m in np.mean(post_samples, axis=0)])
            maps = np.array(['{0:.3f}'.format(estimate_map(post_samples[:, j])) for j in range(len(prior.param_names))])

            # Prepare table
            table = pd.DataFrame(index=prior.param_names, data={
                'Median': meds,
                'Mean': means,
                'MAP': maps,
                '95-CI': qs_95_str
            })

            # Add ground truth parameters to the table
            gt_params = ['{0:.3f}'.format(gt) for gt in ground_truth]
            table['Ground Truth'] = gt_params

            # Save table as CSV
            table.to_csv(resInputs['scnr_nm'] + '_' + str(i) + '_posterior_summary.csv')

            f1 = plot_ppc(
                real_data['summary_conditions'][0, :, :],
                post_sims['sim_data'],
                y_labels=vensimInputs['out_units'],
                titles=vensimInputs['out_labels'],
                cut_end=cut_end
            )
            if resInputs['save_graphs']:
                f1.savefig(resInputs['scnr_nm'] + '_' + str(i) + '_PPC.svg', format='svg', dpi=300)

            f2 = plot_posterior_2d_GT(
                post_samples,
                param_names=prior.param_names,
                params_ground_truth=ground_truth
            )
            if resInputs['save_graphs']:
                f2.savefig(resInputs['scnr_nm'] + '_' + str(i) + '_posteriorEstimate.svg', format='svg', dpi=300)

    print("Ground truth parameters:", ground_truth)

    return runOutputs


In [ ]:
# function for running experiments and collecting data and also saving it
def run_experiments_and_collect_data(base_vensimInputs, base_BFInputs, base_resInputs, experiments, save_path):
    """
    Runs experiments with different conditions and collects outputs in a DataFrame, including full ndarrays.
    
    Parameters:
    - base_vensimInputs: Dictionary with baseline vensim input values.
    - base_BFInputs: Dictionary with baseline BF input values.
    - base_resInputs: Dictionary with baseline res input values.
    - experiments: List of dictionaries specifying the experimental conditions.
    - save_path: Path to save the results DataFrame.
    
    Returns:
    - DataFrame with the results of all experiments.
    """
    all_results = []  # Initialize a list to store experiment results
    
    for exp in experiments:
        # Copy baseline inputs
        vensimInputs = base_vensimInputs.copy()
        BFInputs = base_BFInputs.copy()
        resInputs = base_resInputs.copy()
        
        # Update inputs based on experiment conditions
        vensimInputs.update(exp.get('vensim_changes', {}))
        BFInputs.update(exp.get('BF_changes', {}))
        
        # Run the simulation
        outputs = run_full_analysis(vensimInputs, BFInputs, resInputs)
        
        # Prepare the result dictionary
        result = {'experiment_label': exp['label']}
        result.update(outputs)
        
        # Arrays will be stored directly in the DataFrame
        all_results.append(result)
    
    # Convert the list of dictionaries to a DataFrame
    results_df = pd.DataFrame(all_results)
    
    
    directory = os.path.dirname(save_path)  # Extracts the directory part of the save_path
    
    # Check if the directory exists; if not, create it
    if not os.path.exists(directory):
        os.makedirs(directory)  # This creates the directory and any intermediate directories
        
    # Save the DataFrame to disk for future retrieval
    results_df.to_pickle(save_path)
    
    return results_df



# function for graphing the results of the experiment

def plot_experiment_results(*results_dfs, x_key, y_key, x_label, y_label, df_labels, file_name, title_name):
    """
    Plots the results of experiments as a scatter plot with lines connecting points within each DataFrame,
    with custom axis labels and legend labels for each DataFrame.
    
    Parameters:
    - *results_dfs: Variable number of DataFrame objects containing experiment results.
    - x_key: The key for the column to be plotted on the X-axis.
    - y_key: The key for the column to be plotted on the Y-axis.
    - x_label: Label for the X-axis.
    - y_label: Label for the Y-axis.
    - df_labels: List of strings, labels for each DataFrame corresponding to results_dfs.
    """
    if len(results_dfs) != len(df_labels):
        raise ValueError("The number of DataFrames and df_labels must be the same.")
    
    # Define markers and colors for differentiating between DataFrames
    markers = ['o', 's', '^', 'D', 'x', '>', '<', 'p', '*', '+']  # Extended with more marker types
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'orange', 'purple', 'brown']  # Extended with more colors

    
    plt.figure(figsize=(10, 6))
    
    for df_index, df in enumerate(results_dfs):
        if x_key not in df.columns or y_key not in df.columns:
            print(f"DataFrame {df_index} does not contain the specified keys.")
            continue
        
        x_values = df[x_key]
        y_values = df[y_key]
        labels = df['experiment_label']
        
        plt.scatter(x_values, y_values, label=df_labels[df_index], marker=markers[df_index % len(markers)], color=colors[df_index % len(colors)])
        plt.plot(x_values, y_values, color=colors[df_index % len(colors)])  # Connect points with a line
        
        # Annotate points with labels
        for (x, y, label) in zip(x_values, y_values, labels):
            plt.text(x, y, label, fontsize=9)
    
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.title(title_name)
    plt.savefig(file_name + '.pdf', format='pdf',dpi=300)
    plt.show()




In [ ]:
# This is the main part of code for defining the experiments, running them, and creating the desired graphs
def update_flags(flags, indices, value):
    """
    Update multiple flags at specific indices to either on (1) or off (0).
    
    Args:
    - flags (list): The list of experiment flags.
    - indices (list): The indices of the flags to update.
    - value (int): The new value of the flags (1 for on, 0 for off).
    """
    for index in indices:
        if index < 0 or index >= len(flags):
            print(f"Index {index} out of range.")
            continue
        flags[index] = value
        
exp_flags=[0]*15
#update_flags(exp_flags,[0,1,3,4,5,6,13,14],1)
update_flags(exp_flags,[],1)
load_data=False

'''
experiments:
experiment1_name='Batches'
experiment2_name='lstm_units'
experiment3_name='Batches_offline'
experiment4_name='summary_dim'
experiment5_name='conv_layer'
experiment6_name='coupling_layer'
experiment7_name='learning_rate'
experiment8_name='float_bidir'
experiment9_name='num_heads'
experiment10_name='key_dim'
experiment11_name='att_blocks'
experiment12_name='dens_blocks'
experiment13_name='bidirectional'
experiment14_name='manual_summ'
'''

# update common parameter for offline experiment
offBFInputs=BFInputs
if exp_flags[0]:
    # Define your baseline inputs and experimental conditions
    experiment1_name='Batches'
    experiments = [
        {
            'label': 'B16',
            'vensim_changes': {},
            'BF_changes': {'batch_size' : [16]}
        },
        {
            'label': 'B32',
            'vensim_changes': {},
            'BF_changes': {'batch_size' : [32]}
        },
        {
            'label': 'B64',
            'vensim_changes': {},
            'BF_changes': {'batch_size' : [64]}
        },
        {
            'label': 'B128',
            'vensim_changes': {},
            'BF_changes': {'batch_size' : [128]}
        },
        {
            'label': 'B256',
            'vensim_changes': {},
            'BF_changes': {'batch_size' : [256]}
        },
    ]
    
    # Define the path where you want to save the results
    save_path = experiment1_name+'/results_dataframe' + experiment1_name + '.pkl'

    if load_data:
        results_df_1 = pd.read_pickle(save_path)
    else: 
        # Run the experiments and save the results
        results_df_1 = run_experiments_and_collect_data(vensimInputs, BFInputs, resInputs, experiments, save_path)


# The results are now saved to 'results_dataframe.pkl' and can be loaded in the future
# loaded_results_df = pd.read_pickle('path_to_save/results_dataframe.pkl')

if exp_flags[1]:
    experiment2_name='lstm_units'
    experiments = [

        {
            'label': 'L64',
            'vensim_changes': {},
            'BF_changes': {'lstm_units': 64}
        },
        {
            'label': 'L128',
            'vensim_changes': {},
            'BF_changes': {'lstm_units': 128}
        },        
        {
            'label': 'L256',
            'vensim_changes': {},
            'BF_changes': {'lstm_units': 256}
        },
     
    ]
    
    # Define the path where you want to save the results
    save_path = experiment2_name+'/results_' + experiment2_name + '.pkl'
    if load_data:
        results_df_2 = pd.read_pickle(save_path)
    else: 
        # Run the experiments and save the results
        results_df_2 = run_experiments_and_collect_data(vensimInputs, BFInputs, resInputs, experiments, save_path)



if exp_flags[2]:
    # Define your baseline inputs and experimental conditions
    experiment3_name='Batches_offline'
    experiments = [
        {
            'label': 'OB16',
            'vensim_changes': {},
            'BF_changes': {'batch_size' : [16]}
        },
        {
            'label': 'OB32',
            'vensim_changes': {},
            'BF_changes': {'batch_size' : [32]}
        },
        {
            'label': 'OB64',
            'vensim_changes': {},
            'BF_changes': {'batch_size' : [64]}
        },
        {
            'label': 'OB128',
            'vensim_changes': {},
            'BF_changes': {'batch_size' : [128]}
        },
        {
            'label': 'OB256',
            'vensim_changes': {},
            'BF_changes': {'batch_size' : [256]}
        },
    ]
    
    # Define the path where you want to save the results
    save_path = experiment3_name+'/results_dataframe' + experiment3_name + '.pkl'
      
    if load_data:
        results_df_3 = pd.read_pickle(save_path)
    else:
        # Run the experiments and save the results
        results_df_3 = run_experiments_and_collect_data(vensimInputs, offBFInputs, resInputs, experiments, save_path)

if exp_flags[3]:
    # Define your baseline inputs and experimental conditions
    experiment4_name='summary_dim'
    experiments = [
        {
            'label': 'Sd20',
            'vensim_changes': {},
            'BF_changes': {'summary_dim': 20}
        },
        {
            'label': 'Sd30',
            'vensim_changes': {},
            'BF_changes': {'summary_dim': 30}
        },
        {
            'label': 'Sd50',
            'vensim_changes': {},
            'BF_changes': {'summary_dim': 50}
        },        
    ]
    
    # Define the path where you want to save the results
    save_path = experiment4_name+'/results_dataframe' + experiment4_name + '.pkl'
    
    if load_data:
        results_df_4 = pd.read_pickle(save_path)
    else:
        # Run the experiments and save the results
        results_df_4 = run_experiments_and_collect_data(vensimInputs, BFInputs, resInputs, experiments, save_path)
    
    
    
if exp_flags[4]:
    
    # Define your baseline inputs and experimental conditions
    experiment5_name='conv_layer'
    experiments = [
        {
            'label': 'Cl2',
            'vensim_changes': {},
            'BF_changes': {'num_conv_layers' : 2}
        },
        {
            'label': 'Cl3',
            'vensim_changes': {},
            'BF_changes': {'num_conv_layers' : 3}
        },
        {
            'label': 'Cl4',
            'vensim_changes': {},
            'BF_changes': {'num_conv_layers' : 4}
        }
    ]
    
    # Define the path where you want to save the results
    save_path = experiment5_name+'/results_dataframe' + experiment5_name + '.pkl'
    
    if load_data:
        results_df_5 = pd.read_pickle(save_path)
    else:
        # Run the experiments and save the results
        results_df_5 = run_experiments_and_collect_data(vensimInputs, offBFInputs, resInputs, experiments, save_path)
    
if exp_flags[5]:   
    # Define your baseline inputs and experimental conditions
    experiment6_name='coupling_layer'
    experiments = [
        {
            'label': 'Cp4',
            'vensim_changes': {},
            'BF_changes': {'num_coupling_layers' : 4}
        },
        {
            'label': 'Cp6',
            'vensim_changes': {},
            'BF_changes': {'num_coupling_layers' : 6}
        },
        {
            'label': 'Cp8',
            'vensim_changes': {},
            'BF_changes': {'num_coupling_layers' : 8}
        }
    ]
    
    # Define the path where you want to save the results
    save_path = experiment6_name+'/results_dataframe' + experiment6_name + '.pkl'
    
    if load_data:
        results_df_6 = pd.read_pickle(save_path)
    else:
        # Run the experiments and save the results
        results_df_6 = run_experiments_and_collect_data(vensimInputs, offBFInputs, resInputs, experiments, save_path)
    
if exp_flags[6]:   
    
    # Define your baseline inputs and experimental conditions
    experiment7_name='learning_rate'
    experiments = [
        {
            'label': 'LR20',
            'vensim_changes': {},
            'BF_changes': {'learning_rate': 0.0020}
        },
        {
            'label': 'LR10',
            'vensim_changes': {},
            'BF_changes': {'learning_rate': 0.0010}
        },
        {
            'label': 'LR5',
            'vensim_changes': {},
            'BF_changes': {'learning_rate': 0.0005}
        },
    ]
    
    # Define the path where you want to save the results
    save_path = experiment7_name+'/results_dataframe' + experiment7_name + '.pkl'
    
    if load_data:
        results_df_7 = pd.read_pickle(save_path)
    else:
        # Run the experiments and save the results
        results_df_7 = run_experiments_and_collect_data(vensimInputs, offBFInputs, resInputs, experiments, save_path)
    
if exp_flags[7]:    
    
    # Define your baseline inputs and experimental conditions
    experiment8_name='float_bidir'
    experiments = [
        {
            'label': 'f32b',
            'vensim_changes': {},
            'BF_changes': {'bidirectional': True}
        },
        {
            'label': 'f16b',
            'vensim_changes': {},
            'BF_changes': {'float16' : True, 'bidirectional': True}
        },
        {
            'label': 'f32u',
            'vensim_changes': {},
            'BF_changes': {'bidirectional': False}
        },
        {
            'label': 'f16u',
            'vensim_changes': {},
            'BF_changes': {'float16' : True, 'bidirectional': False}
        },
    ]
    
    # Define the path where you want to save the results
    save_path = experiment8_name+'/results_dataframe' + experiment8_name + '.pkl'
    
    if load_data:
        results_df_8 = pd.read_pickle(save_path)
    else:
        # Run the experiments and save the results
        results_df_8 = run_experiments_and_collect_data(vensimInputs, offBFInputs, resInputs, experiments, save_path)
    
if exp_flags[8]:    
    
    # Define your baseline inputs and experimental conditions
    experiment9_name='num_heads'
    experiments = [
        {
            'label': 'h2',
            'vensim_changes': {},
            'BF_changes': {'num_heads': 2}
        },
        {
            'label': 'h4-base',
            'vensim_changes': {},
            'BF_changes': {'num_heads': 4}
        },
        {
            'label': 'h8',
            'vensim_changes': {},
            'BF_changes': {'num_heads': 8}
        },

    ]
    
    # Define the path where you want to save the results
    save_path = experiment9_name+'/results_dataframe' + experiment9_name + '.pkl'
    
    if load_data:
        results_df_9 = pd.read_pickle(save_path)
    else:
        # Run the experiments and save the results
        results_df_9 = run_experiments_and_collect_data(vensimInputs, BFInputs, resInputs, experiments, save_path)

if exp_flags[9]:    
    
    # Define your baseline inputs and experimental conditions
    experiment10_name='key_dim'
    experiments = [
        {
            'label': 'k16',
            'vensim_changes': {},
            'BF_changes': {'key_dim': 16}
        },
        {
            'label': 'k64',
            'vensim_changes': {},
            'BF_changes': {'key_dim': 64}
        },

    ]
    
    # Define the path where you want to save the results
    save_path = experiment10_name+'/results_dataframe' + experiment10_name + '.pkl'
    
    if load_data:
        results_df_10 = pd.read_pickle(save_path)
    else:
        # Run the experiments and save the results
        results_df_10 = run_experiments_and_collect_data(vensimInputs, BFInputs, resInputs, experiments, save_path)

if exp_flags[10]:    
    
    # Define your baseline inputs and experimental conditions
    experiment11_name='att_blocks'
    experiments = [
        {
            'label': 'ab3',
            'vensim_changes': {},
            'BF_changes': {'num_attention_blocks': 3}
        },

    ]
    
    # Define the path where you want to save the results
    save_path = experiment11_name+'/results_dataframe' + experiment11_name + '.pkl'
    
    if load_data:
        results_df_11 = pd.read_pickle(save_path)
    else:
        # Run the experiments and save the results
        results_df_11 = run_experiments_and_collect_data(vensimInputs, BFInputs, resInputs, experiments, save_path)

if exp_flags[11]:    
    
    # Define your baseline inputs and experimental conditions
    experiment12_name='dens_blocks'
    experiments = [
        {
            'label': 'db3',
            'vensim_changes': {},
            'BF_changes': {'num_dense': 3}
        },

    ]
    
    # Define the path where you want to save the results
    save_path = experiment12_name+'/results_dataframe' + experiment12_name + '.pkl'
    
    if load_data:
        results_df_12 = pd.read_pickle(save_path)
    else:
        # Run the experiments and save the results
        results_df_12 = run_experiments_and_collect_data(vensimInputs, BFInputs, resInputs, experiments, save_path)

if exp_flags[12]:    
    
    # Define your baseline inputs and experimental conditions
    experiment13_name='bidirectional'
    experiments = [
        {
            'label': 'uni_dir',
            'vensim_changes': {},
            'BF_changes': {'bidirectional': False}
        },

    ]
    
    # Define the path where you want to save the results
    save_path = experiment13_name+'/results_dataframe' + experiment13_name + '.pkl'
    
    if load_data:
        results_df_13 = pd.read_pickle(save_path)
    else:
        # Run the experiments and save the results
        results_df_13 = run_experiments_and_collect_data(vensimInputs, BFInputs, resInputs, experiments, save_path)

if exp_flags[13]:    
    
    # Define your baseline inputs and experimental conditions
    experiment14_name='manual_summ'
    experiments = [
        {
            'label': '+man',
            'vensim_changes': {},
            'BF_changes': {'manual_sum_stats': True}
        },
        {
            'label': 'no_man',
            'vensim_changes': {},
            'BF_changes': {'manual_sum_stats': False}
        },

    ]
    
    # Define the path where you want to save the results
    save_path = experiment14_name+'/results_dataframe' + experiment14_name + '.pkl'
    
    if load_data:
        results_df_14 = pd.read_pickle(save_path)
    else:
        # Run the experiments and save the results
        results_df_14 = run_experiments_and_collect_data(vensimInputs, BFInputs, resInputs, experiments, save_path)


if exp_flags[14]:    
    
    # Define your baseline inputs and experimental conditions
    experiment15_name='epochs'
    experiments = [
        {
            'label': 'e10',
            'vensim_changes': {},
            'BF_changes': {'epochs': 10}
        },
        {
            'label': 'e20',
            'vensim_changes': {},
            'BF_changes': {'epochs': 20}
        },
        {
            'label': 'e30',
            'vensim_changes': {},
            'BF_changes': {'epochs': 30}
        },
    ]
    
    # Define the path where you want to save the results
    save_path = experiment15_name+'/results_dataframe' + experiment15_name + '.pkl'
    
    if load_data:
        results_df_15 = pd.read_pickle(save_path)
    else:
        # Run the experiments and save the results
        results_df_15 = run_experiments_and_collect_data(vensimInputs, BFInputs, resInputs, experiments, save_path)



In [ ]:
# Doing all the graphs

'''
experiments:
experiment1_name='Batches'
experiment2_name='lstm_units'
experiment3_name='Batches_offline'
experiment4_name='summary_dim'
experiment5_name='conv_layer'
experiment6_name='coupling_layer'
experiment7_name='learning_rate'
experiment8_name='float_bidir'
experiment9_name='num_heads'
experiment10_name='key_dim'
experiment11_name='att_blocks'
experiment12_name='dens_blocks'
experiment13_name='bidirectional'
experiment14_name='manual_summ'
'''

'''
plot_experiment_results(results_df_1, results_df_2, results_df_3, results_df_4, x_key='train_time', y_key='val_loss', x_label='Training Time (seconds)',y_label='Loss Values',
                       df_labels=['Batch Sizes','LSTM Units','Offline Batches','Summary Dimensions'],
                        file_name = 'batch_LSTM_Summ_RandomWalk.pdf',title_name = 'Random Walk Hyper Parameters (Round Based)')


plot_experiment_results(results_df_5, results_df_6, results_df_7, results_df_8, x_key='train_time', y_key='val_loss', x_label='Training Time (seconds)',y_label='Loss Values',
                       df_labels=['Convolution Layers','Coupling Layers','Learning Rates','Float and bidirectional'],
                        file_name = 'NN_Learning_float_RandomWalk',title_name = 'Random Walk Hyper Parameters (offline)')


plot_experiment_results(results_df_2, results_df_9, results_df_10, results_df_11,results_df_12, results_df_13, x_key='train_time', y_key='val_loss', x_label='Training Time (seconds)',y_label='Loss Values',
                       df_labels=['LSTM Units','Number of Heads','Key Dim','Attention Blocks','Dense Blocks','Bidirectional'],
                        file_name = 'NN_Transformer_RandomWalk',title_name = 'Random Walk Transformer:500*7R*5E,32B')
                      
plot_experiment_results(results_df_14, x_key='train_time', y_key='val_loss', x_label='Training Time (seconds)',y_label='Loss Values',
                       df_labels=['Manual Summary Stats'],
                        file_name = 'NN_RandomWalk_manual',title_name = 'Random Walk Manual:500*5R*5E,32B')
'''

'''
plot_experiment_results(results_df_1, x_key='train_time', y_key='val_loss', x_label='Training Time (seconds)',y_label='Loss Values',
                       df_labels=['Batches'],
                        file_name = 'Batches',title_name = 'Random Walk Hyper Parameters (Round Based)')

plot_experiment_results(results_df_2, x_key='train_time', y_key='val_loss', x_label='Training Time (seconds)',y_label='Loss Values',
                       df_labels=['LSTM Units'],
                        file_name = 'LSTM Units',title_name = 'Random Walk Hyper Parameters (Round Based)')
                        
plot_experiment_results(results_df_4, x_key='train_time', y_key='val_loss', x_label='Training Time (seconds)',y_label='Loss Values',
                       df_labels=['summary_dim'],
                        file_name = 'Summary Dimensions',title_name = 'Random Walk Hyper Parameters (Round Based)')
                        
plot_experiment_results(results_df_5, x_key='train_time', y_key='val_loss', x_label='Training Time (seconds)',y_label='Loss Values',
                       df_labels=['num_conv_layers'],
                        file_name = 'Convolutional Layers',title_name = 'Random Walk Hyper Parameters (Round Based)')

plot_experiment_results(results_df_6, x_key='train_time', y_key='val_loss', x_label='Training Time (seconds)',y_label='Loss Values',
                       df_labels=['num_coupling_layers'],
                        file_name = 'Coupling Layers',title_name = 'Random Walk Hyper Parameters (Round Based)')



plot_experiment_results(results_df_7, x_key='train_time', y_key='val_loss', x_label='Training Time (seconds)',y_label='Loss Values',
                       df_labels=['learning_rate'],
                        file_name = 'Learning Rate',title_name = 'Random Walk Hyper Parameters (Round Based)')




plot_experiment_results(results_df_8, x_key='train_time', y_key='val_loss', x_label='Training Time (seconds)',y_label='Loss Values',
                       df_labels=['float_bidir'],
                        file_name = 'Float & Bidirectional',title_name = 'Random Walk Hyper Parameters (Round Based)')



plot_experiment_results(results_df_14, x_key='train_time', y_key='val_loss', x_label='Training Time (seconds)',y_label='Loss Values',
                       df_labels=['manual_sum_stats'],
                        file_name = 'Manual Summary Statistics',title_name = 'Random Walk Hyper Parameters (Round Based)')

'''                        

In [ ]:
# an optimized run:

runOptimized = True
if runOptimized:
    BFInputsOpt=BFInputs
    #BFInputsOpt['epochs']=50
    BFInputsOpt['epochs']=10
    BFInputsOpt['batch_size']=[32]
    BFInputsOpt['learning_rate']=0.0010
    #BFInputsOpt['num_coupling_layers']= 6
    BFInputsOpt['num_coupling_layers']= 6
    BFInputsOpt['lstm_units']= 128
    BFInputsOpt['summary_dim']= 20
    BFInputsOpt['num_conv_layers']=4
    BFInputsOpt['sum_network_type']='sequence'
    BFInputsOpt['bidirectional']=True
    BFInputsOpt['num_rounds']=20
    #BFInputsOpt['sims_per_round']= 131072
    BFInputsOpt['sims_per_round']= 8192
    BFInputsOpt['manual_sum_stats'] = True

    outputsOptTrans = run_full_analysis(vensimInputs, BFInputsOpt, resInputs)
    
    print(outputsOptTrans)
  
